# sPyTial for Data Structures: Making Code Spatial

**Discover how sPyTial transforms traditional data structure debugging and visualization**

This notebook demonstrates sPyTial's core capability: revealing the spatial relationships inherent in your data structures. Instead of text-based output or complex debugging sessions, see your data's architecture visually.

## Why Spatial Visualization for Data Structures?

**Traditional debugging:**
```python
def print_tree(node, level=0):
    if node:
        print("  " * level + str(node.value))
        print_tree(node.left, level + 1)
        print_tree(node.right, level + 1)
```
Output: Text that doesn't scale and loses structure

**sPyTial approach:**
```python
@orientation(selector='{ x : TreeNode, y : TreeNode | x.left = y}', directions=['below', 'left'])
class TreeNode:
    # ... automatically visualizes spatial relationships
```
Output: Interactive spatial diagram showing actual tree structure

In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import diagram
from spytial.annotations import orientation, attribute, hideAtom, atomColor, group
from collections import namedtuple, deque
import json

## 1. Basic Data Structures: Beyond Print Statements

Let's start with common Python data structures and see how sPyTial reveals their organization.

In [2]:
# Traditional approach: nested dictionaries and lists
application_data = {
    "users": [
        {"name": "Alice", "age": 30, "skills": ["Python", "JavaScript"]},
        {"name": "Bob", "age": 25, "skills": ["Java", "C++"]}
    ],
    "projects": {
        "web_app": {"lead": "Alice", "status": "active"},
        "mobile_app": {"lead": "Bob", "status": "planning"}
    },
    "stats": {"total_users": 2, "active_projects": 1}
}

print("Traditional output (try to see the structure):")
print(json.dumps(application_data, indent=2)[:200] + "...")

print("\nsPyTial spatial view:")
diagram(application_data)

Traditional output (try to see the structure):
{
  "users": [
    {
      "name": "Alice",
      "age": 30,
      "skills": [
        "Python",
        "JavaScript"
      ]
    },
    {
      "name": "Bob",
      "age": 25,
      "skills": [
     ...

sPyTial spatial view:


## 2. Binary Trees: From ASCII Art to Spatial Relations

Binary trees are perfect for demonstrating sPyTial's power. Traditional approaches use ASCII art or recursive printing that breaks down with larger trees.

In [3]:
# Define a spatially-aware binary tree
@orientation(selector='{ x : TreeNode, y : TreeNode | x.left = y}', directions=['below', 'left'])
@orientation(selector='{ x : TreeNode, y : TreeNode | x.right = y}', directions=['below', 'right'])
@attribute(field='value')
@hideAtom(selector='NoneType')
class TreeNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def __repr__(self):
        return f"TreeNode({self.value})"
    
    def traditional_print(self, level=0):
        """Traditional ASCII tree printing"""
        if self.right:
            self.right.traditional_print(level + 1)
        print('    ' * level + str(self.value))
        if self.left:
            self.left.traditional_print(level + 1)
    
# Create a sample binary search tree
root = TreeNode(
    value=10,
    left=TreeNode(
        value=5,
        left=TreeNode(3),
        right=TreeNode(7)
    ),
    right=TreeNode(
        value=15,
        left=TreeNode(12),
        right=TreeNode(18)
    )
)

print("Traditional ASCII output:")
root.traditional_print()

print("\nsPyTial spatial visualization:")
diagram(root)

Traditional ASCII output:
        18
    15
        12
10
        7
    5
        3

sPyTial spatial visualization:


## 3. Sets and Collections: Revealing Hidden Structure

Sets traditionally appear as unordered collections. sPyTial can reveal their internal organization and relationships.

In [4]:
# Traditional set visualization
numbers = {1, 22, 3, 4, 55, 6, 7, 8}
print(f"Traditional set output: {numbers}")
print("No structure visible, just comma-separated values")

print("\nsPyTial set visualization:")
diagram(numbers)

# Group elements to show containment relationships more clearly
print("\nWith grouping to emphasize set containment:")
grouped_set = group(field='contains', groupOn=0, addToGroup=1)(numbers)
diagram(grouped_set)

Traditional set output: {1, 3, 4, 6, 7, 8, 22, 55}
No structure visible, just comma-separated values

sPyTial set visualization:



With grouping to emphasize set containment:


## 4. Queues and Stacks: Visualizing Linear Structures

Linear data structures like queues and stacks benefit from spatial layout that shows their FIFO/LIFO nature.

In [5]:
# Queue simulation using deque
from collections import deque

# Create a task queue
task_queue = deque(["login_user", "process_payment", "send_email", "update_inventory"])
print(f"Traditional queue output: {list(task_queue)}")
print("Just a list - no sense of FIFO ordering")

print("\nsPyTial queue visualization:")
diagram(task_queue)

# Stack simulation using list
function_call_stack = ["main()", "process_order()", "validate_payment()", "check_inventory()"]
print(f"\nTraditional stack output: {function_call_stack}")
print("\nsPyTial stack visualization:")
diagram(function_call_stack)

Traditional queue output: ['login_user', 'process_payment', 'send_email', 'update_inventory']
Just a list - no sense of FIFO ordering

sPyTial queue visualization:



Traditional stack output: ['main()', 'process_order()', 'validate_payment()', 'check_inventory()']

sPyTial stack visualization:


## 5. Graph Structures: Connected Data Relationships

Graphs are inherently spatial but traditionally visualized through adjacency lists or matrices. sPyTial shows actual connections.

In [6]:
# Simple graph using dictionaries
social_network = {
    "Alice": ["Bob", "Charlie"],
    "Bob": ["Alice", "David", "Eve"],
    "Charlie": ["Alice", "Eve"],
    "David": ["Bob"],
    "Eve": ["Bob", "Charlie"]
}

print("Traditional adjacency list:")
for person, friends in social_network.items():
    print(f"{person}: {friends}")

print("\nsPyTial graph visualization:")
diagram(social_network)

# More complex graph with weighted edges
route_network = {
    "NYC": {"Boston": 215, "DC": 230, "Philadelphia": 95},
    "Boston": {"NYC": 215, "Portland": 108},
    "DC": {"NYC": 230, "Atlanta": 640},
    "Philadelphia": {"NYC": 95, "DC": 140},
    "Portland": {"Boston": 108},
    "Atlanta": {"DC": 640}
}

print("\nWeighted route network:")
diagram(route_network)

Traditional adjacency list:
Alice: ['Bob', 'Charlie']
Bob: ['Alice', 'David', 'Eve']
Charlie: ['Alice', 'Eve']
David: ['Bob']
Eve: ['Bob', 'Charlie']

sPyTial graph visualization:



Weighted route network:


## 6. Custom Data Structures: Domain-Specific Visualization

sPyTial excels at visualizing domain-specific data structures by revealing their inherent spatial relationships.

In [7]:
# File system hierarchy
@orientation(selector='{ x : Directory, y : FileSystemItem | x.contains = y}', directions=['below'])
class Directory:
    def __init__(self, name, items=None):
        self.name = name
        self.contains = items or []
    
    def __repr__(self):
        return f"📁 {self.name}/"

class File:
    def __init__(self, name, size):
        self.name = name
        self.size = size
    
    def __repr__(self):
        return f"📄 {self.name} ({self.size}KB)"

# Create a file system structure
project_dir = Directory("my_project", [
    Directory("src", [
        File("main.py", 15),
        File("utils.py", 8),
        Directory("models", [
            File("user.py", 12),
            File("product.py", 9)
        ])
    ]),
    Directory("tests", [
        File("test_main.py", 6),
        File("test_utils.py", 4)
    ]),
    File("README.md", 3),
    File("requirements.txt", 1)
])

print("Traditional directory listing:")
def print_tree(item, level=0):
    indent = "  " * level
    print(f"{indent}{item}")
    if hasattr(item, 'contains'):
        for sub_item in item.contains:
            print_tree(sub_item, level + 1)

print_tree(project_dir)

print("\nsPyTial file system visualization:")
diagram(project_dir)

Traditional directory listing:
📁 my_project/
  📁 src/
    📄 main.py (15KB)
    📄 utils.py (8KB)
    📁 models/
      📄 user.py (12KB)
      📄 product.py (9KB)
  📁 tests/
    📄 test_main.py (6KB)
    📄 test_utils.py (4KB)
  📄 README.md (3KB)
  📄 requirements.txt (1KB)

sPyTial file system visualization:


## 7. Advanced Trees: Red-Black Trees with Color Coding

Complex data structures like Red-Black trees have additional properties that traditional visualization can't easily show.

In [8]:
# Red-Black Tree with color annotations
@orientation(selector='{ x : RBTreeNode, y : RBTreeNode | x.left = y}', directions=['below', 'left'])
@orientation(selector='{ x : RBTreeNode, y : RBTreeNode | x.right = y}', directions=['below', 'right'])
@atomColor(selector='{ x : RBTreeNode | x.color = "red" }', value='red')
@atomColor(selector='{ x : RBTreeNode | x.color = "black" }', value='black')
@attribute(field='value')
class RBTreeNode:
    def __init__(self, value, color, left=None, right=None):
        self.value = value
        self.color = color
        self.left = left
        self.right = right

    def __repr__(self):
        return f"RBTreeNode({self.value}, {self.color})"
    
    def traditional_print(self, level=0):
        """Traditional colored tree printing"""
        if self.right:
            self.right.traditional_print(level + 1)
        color_marker = "(R)" if self.color == "red" else "(B)"
        print('    ' * level + f"{self.value}{color_marker}")
        if self.left:
            self.left.traditional_print(level + 1)

# Create a Red-Black Tree
rb_root = RBTreeNode(
    value=10, color="black",
    left=RBTreeNode(
        value=5, color="red",
        left=RBTreeNode(3, "black"),
        right=RBTreeNode(7, "black")
    ),
    right=RBTreeNode(
        value=15, color="red",
        left=RBTreeNode(12, "black"),
        right=RBTreeNode(18, "black")
    )
)

print("Traditional Red-Black tree output:")
rb_root.traditional_print()
print("(R) = Red, (B) = Black")

print("\nsPyTial Red-Black tree with color coding:")
diagram(rb_root)

Traditional Red-Black tree output:
        18(B)
    15(R)
        12(B)
10(B)
        7(B)
    5(R)
        3(B)
(R) = Red, (B) = Black

sPyTial Red-Black tree with color coding:


## 8. Comparative Analysis: Traditional vs Spatial

Let's directly compare traditional debugging approaches with sPyTial's spatial visualization for a complex nested structure.

In [10]:
# Complex nested application state
app_state = {
    "authentication": {
        "current_user": {"id": 123, "name": "Alice", "role": "admin"},
        "session": {"token": "abc123", "expires": "2024-02-01"},
        "permissions": ["read", "write", "delete"]
    },
    "data": {
        "cache": {"users": ["Alice", "Bob"], "products": ["laptop", "mouse"]},
        "pending_requests": deque(["req1", "req2", "req3"]),
        "active_connections": {"websocket": 5, "http": 12}
    },
    "ui_state": {
        "current_page": "dashboard",
        "sidebar_open": True,
        "notifications": [{"type": "info", "message": "Welcome"}, {"type": "warning", "message": "Update required"}]
    }
}

print("Traditional debugging approach:")
print("1. Print entire structure (overwhelming):")
print(json.dumps(app_state, indent=2, default=str)[:300] + "...")

print("\n2. Navigate manually (time-consuming):")
print(f"Current user: {app_state['authentication']['current_user']['name']}")
print(f"Cache size: {len(app_state['data']['cache'])}")
print(f"Notifications: {len(app_state['ui_state']['notifications'])}")

print("\n3. Write custom traversal functions (repetitive):")
def print_structure(obj, level=0):
    indent = "  " * level
    if isinstance(obj, dict):
        for key, value in obj.items():
            if isinstance(value, (dict, list)):
                print(f"{indent}{key}: <{type(value).__name__}>")
            else:
                print(f"{indent}{key}: {value}")
print_structure(app_state)

print("\n" + "="*50)
print("sPyTial approach - immediate spatial understanding:")
diagram(app_state)

Traditional debugging approach:
1. Print entire structure (overwhelming):
{
  "authentication": {
    "current_user": {
      "id": 123,
      "name": "Alice",
      "role": "admin"
    },
    "session": {
      "token": "abc123",
      "expires": "2024-02-01"
    },
    "permissions": [
      "read",
      "write",
      "delete"
    ]
  },
  "data": {
    "cache": {
   ...

2. Navigate manually (time-consuming):
Current user: Alice
Cache size: 2
Notifications: 2

3. Write custom traversal functions (repetitive):
authentication: <dict>
data: <dict>
ui_state: <dict>

sPyTial approach - immediate spatial understanding:


## Key Takeaways: Why sPyTial Transforms Data Structure Work

### Traditional Limitations:
- **Text output doesn't scale** - becomes unreadable with complexity
- **No spatial intuition** - relationships hidden in syntax
- **Manual traversal required** - time-consuming debugging
- **Static representation** - can't interact or explore

### sPyTial Advantages:
- **Immediate visual comprehension** - see structure at a glance
- **Spatial relationships revealed** - understand connections naturally
- **Scales with complexity** - large structures remain comprehensible
- **Interactive exploration** - zoom, pan, and investigate details
- **Annotation support** - add semantic meaning to visualization

### Best Use Cases:
1. **Debugging complex nested structures** - JSON APIs, configuration objects
2. **Algorithm development** - tree traversal, graph algorithms
3. **Data structure education** - teaching and learning
4. **Code review** - quickly understand data organization
5. **Performance analysis** - identify structural bottlenecks

**Next steps:**
- Explore **00-demo-showcase.ipynb** for overview of all sPyTial capabilities
- See **05-enhanced-data-structures.ipynb** for advanced patterns
- Try **06-provider-development.ipynb** to extend sPyTial for your data types
- Check **02-object-annotations.ipynb** for custom annotation techniques